**Goals**
1. Extract stockdata from AlphaVantage API using a url
2. Extract stock data from AlphaVantage API using a HTTP request
3. Write functions for transforming data

**Import the necessary libraries**

In [1]:
import pandas as pd
import requests

# **Accesing APIs Through URL**

In [2]:
url = (
    "https://www.alphavantage.co/query?"
    "function=TIME_SERIES_DAILY&"
    "symbol=IBM&"
    "apikey=demo"
)

In [3]:
print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo'

**Storing API keys**

In [4]:
alpha_api_key = "PGPVDW57277TSFM3"

**URL for Alibaba Group Holding Limited**

In [5]:
url = (
    "https://www.alphavantage.co/query?"
    "function=TIME_SERIES_DAILY&"
    "symbol=BABA&"
    f"apikey={alpha_api_key}"
)
print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=BABA&apikey=PGPVDW57277TSFM3'

**URL for Alibaba Group Holdings Limited With Parameters**

In [6]:
ticker = "BABA"
output_size = "compact"
data_type = "json"

url = (
    "https://www.alphavantage.co/query?"
    "function=TIME_SERIES_DAILY&"
    f"symbol={ticker}&"
    f"outputsize={output_size}&"
    f"datatype={data_type}&"
    f"apikey={alpha_api_key}"
)
print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=BABA&outputsize=compact&datatype=json&apikey=PGPVDW57277TSFM3'

# **Accesing APIs through a Request**

In [7]:
response = requests.get(url=url)
print("response type:", type(response))

response type: <class 'requests.models.Response'>


**Response Attributes**

In [8]:
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

**Response Status Code**

In [9]:
response_code = response.status_code
response_code

200

**Response JSON**

In [10]:
response_data = response.json()
print("response_data type:", type(response_data))

response_data type: <class 'dict'>


**Response Data Keys**

In [11]:
# Print "response" data keys
response_data.keys()

dict_keys(['Meta Data', 'Time Series (Daily)'])

**Stock Data**

In [12]:
# Extract "Time Series data" from response_data
stock_data = response_data["Time Series (Daily)"]

# Put stock data to dataframe
df_alibaba = pd.DataFrame.from_dict(stock_data, orient="index", dtype="float")

df_alibaba.head()

,1. open,2. high,3. low,4. close,5. volume
2024-09-03,81.780,83.0200,81.700,82.27,11638188.0
2024-08-30,84.720,84.7500,82.580,83.34,20027304.0
2024-08-29,81.240,81.4650,80.460,81.02,14154155.0
2024-08-28,81.125,81.2799,79.205,79.62,14024406.0
2024-08-27,82.270,82.6500,81.270,81.45,9026068.0


**Datetime Index**

Changing the data type of df_alibaba index from a float to datetime would be appropriate

In [13]:
df_alibaba.index = pd.to_datetime(df_alibaba.index)
# Name index "date"
df_alibaba.index.name = "date"

df_alibaba.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2024-09-03,81.780,83.0200,81.700,82.27,11638188.0
2024-08-30,84.720,84.7500,82.580,83.34,20027304.0
2024-08-29,81.240,81.4650,80.460,81.02,14154155.0
2024-08-28,81.125,81.2799,79.205,79.62,14024406.0
2024-08-27,82.270,82.6500,81.270,81.45,9026068.0


**Column Cleaning**

Lets remove numbering from df_alibaba column names.

In [14]:
df_alibaba.columns = [c.split(".")[1] for c in df_alibaba.columns]
df_alibaba.head()

,open,high,low,close,volume
date,,,,,
2024-09-03,81.780,83.0200,81.700,82.27,11638188.0
2024-08-30,84.720,84.7500,82.580,83.34,20027304.0
2024-08-29,81.240,81.4650,80.460,81.02,14154155.0
2024-08-28,81.125,81.2799,79.205,79.62,14024406.0
2024-08-27,82.270,82.6500,81.270,81.45,9026068.0


**Get Daily Function**

In [15]:
def get_daily(ticker, output_size="full"):
    """Get daily time series of an equity from AlphaVantage.
    parameters
    _ _ _ _ _ _
    ticker: str
        The ticker symbol of the equity.
    output_size: str
        Number of observations to return. "full" means the historic observations of the equtiy to be returned, "compact means the most recent observations."
    returns
    _ _ _ _ _ _
    pd.DataFrame:
        Columns are "open, high, low, close, volume"
    """
    
    # Create URL
    url = (
        "https://www.alphavantage.co/query?"
        "function=TIME_SERIES_DAILY&"
        f"symbol={ticker}&"
        f"outputsize={output_size}&"
        f"datatype=json&"
        f"apikey={alpha_api_key}"
    )
    
    # Send request to api
    response = requests.get(url=url)
    
    # Check if the request  is successfull
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code}")
    
    # Check if the "Time Series key exists" in response_data
    if "Time Series (Daily)" not in response_data.keys():
        raise Exception(
            f"Invalid API call. Check the ticker symbol: {ticker}"
        )
    
    # Extract json data from response
    data = response_data["Time Series (Daily)"]
    
    # Read data into DataFrame
    stock_data = pd.DataFrame.from_dict(data, orient="index", dtype="float")
    
    # Convert "index" to "datetime index" and name "date"
    stock_data.index =  pd.to_datetime(stock_data.index)
    stock_data.index.name = "date"

    # Removing numbering from your column names
    stock_data.columns = [c.split(".")[1] for c in stock_data.columns]
    
    return stock_data

**Test get_daily function**

In [16]:
alibaba_stock_data = get_daily(ticker="AMBUJACEM.BSE")
alibaba_stock_data.head()

,open,high,low,close,volume
date,,,,,
2024-09-03,81.780,83.0200,81.700,82.27,11638188.0
2024-08-30,84.720,84.7500,82.580,83.34,20027304.0
2024-08-29,81.240,81.4650,80.460,81.02,14154155.0
2024-08-28,81.125,81.2799,79.205,79.62,14024406.0
2024-08-27,82.270,82.6500,81.270,81.45,9026068.0
